In [8]:
import fire
import mlflow
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [3]:
def split_data(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1522)
    y_train = y_train.values.ravel()
    return X_train, X_test, y_train, y_test

In [5]:
# we use scikit-learn pipeline to package standarization into single object with model
def scale_pipeline(scaler_name, scaler, model_name, model):
    steps = [(scaler_name, scaler), (model_name, model)]
    pipeline_model = Pipeline(steps)
    return pipeline_model

In [9]:
def randomized_search(pipe, x_train, y_train):
    param_grid = {
              "logreg__penalty" : ["l2"],
              "logreg__tol" : np.linspace(0.0001, 1, 50),
              "logreg__C" : np.linspace(0.1, 1, 50)
             }
    randomized_search_cv = RandomizedSearchCV(pipe, param_grid, cv=10, n_iter=10)
    randomized_search_cv.fit(x_train, y_train)
    best_params = randomized_search_cv.best_params_
    best_pipeline = randomized_search_cv.best_estimator_
    return best_pipeline, best_params

In [10]:
def track_with_mlflow(model, X_test, Y_test, mlflow, model_metadata):
    mlflow.log_params(model_metadata)
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(Y_test, y_pred)
    f1 = f1_score(Y_test, y_pred, average="weighted")
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("F1 Score", f1)
    
    mlflow.sklearn.log_model(model, "logreg", registered_model_name="sklearn_logreg")

In [ ]:
def main(logreg_type: str, solver_name: str):
    wine = datasets.load_wine()
    X = pd.DataFrame(wine.data, columns = wine.feature_names)   # dataframe with all feature columns
    y = pd.DataFrame(wine.target, columns = ['encoded_class'])   # dataframe with target column
    X = X.iloc[:-3]  #?
    y = y.iloc[:-3]  #?

    X_train, X_test, y_train, y_test = split_data(X, y)    
    
    with mlflow.start_run():
        log_model = LogisticRegression(multi_class=logreg_type, solver=solver_name, random_state=1522)
        pipeline = scale_pipeline("scaler", StandardScaler(), "logreg", log_model)
        best_pipe, model_metadata = randomized_search(pipeline, X_train, y_train)
        best_pipe.fit(X_train, y_train)
        y_pred = best_pipe.predict(X_test)
        
        track_with_mlflow(best_pipe, X_test, y_test, mlflow, model_metadata)
    
    
      
#     # let's check some other k
#     k_list = range(1, max_k)

#     for k in k_list:
#         with mlflow.start_run():
#             knn_pipe = setup_knn_pipeline(k)
#             knn_pipe.fit(X_train, Y_train)
#             model_metadata = {"k": k}
#             track_with_mlflow(knn_pipe, X_test, Y_test, mlflow, model_metadata)

In [ ]:
if __name__ == "__main__":
    fire.Fire(main)